In [3]:
export_folder = "pros1"
filename_prefix = "pros1"
sequence_browser_name = "UsScan1"

from local_vars import root_folder
import os

export_fullpath = os.path.join(root_folder, export_folder)

if not os.path.exists(export_fullpath):
    os.makedirs(export_fullpath)
    print("Created folder: " + export_fullpath)

print "Export data to: " + export_fullpath

Export data to: c:\Temp\pros1


In [4]:
segmentation_name = "CatheterSegmentation"
image_name = "Image_Image"

segmentation_node = slicer.util.getFirstNodeByName(segmentation_name, className="vtkMRMLSegmentationNode")
print "Segmentation node: " + segmentation_node.GetName()

image_node = slicer.util.getFirstNodeByName(image_name, className="vtkMRMLScalarVolumeNode")
print "Ultrasound image node: " + image_node.GetName()

# sequence_browser_node = slicer.util.getFirstNodeByName('', className='vtkMRMLSequenceBrowserNode')
sequence_browser_node = slicer.util.getFirstNodeByName(sequence_browser_name, className='vtkMRMLSequenceBrowserNode')
print "Sequence browser node: " + str(sequence_browser_node.GetID())

Segmentation node: CatheterSegmentation
Ultrasound image node: Image_Image
Sequence browser node: vtkMRMLSequenceBrowserNode2


In [5]:
num_items = sequence_browser_node.GetNumberOfItems()
n = num_items
sequence_browser_node.SelectFirstItem()

print "Number of images: " + str(n)

Number of images: 523


In [6]:
labelmap_volume_node = slicer.mrmlScene.AddNewNodeByClass('vtkMRMLLabelMapVolumeNode')

ic = vtk.vtkImageCast()
ic.SetOutputScalarTypeToUnsignedChar()
ic.Update()

png_writer = vtk.vtkPNGWriter()

for i in range(n):
    slicer.modules.segmentations.logic().ExportVisibleSegmentsToLabelmapNode(segmentation_node, labelmap_volume_node, image_node)
    segmented_image = labelmap_volume_node.GetImageData()
    input_image = image_node.GetImageData()
    
    seg_file_name = filename_prefix + "_%04d_segmentation" % i + ".png"
    img_file_name = filename_prefix + "_%04d_ultrasound" % i + ".png"
    seg_fullname = os.path.join(export_fullpath, seg_file_name)
    img_fullname = os.path.join(export_fullpath, img_file_name)
    
    ic.SetInputData(segmented_image)
    ic.Update()
    png_writer.SetInputData(ic.GetOutput())
    png_writer.SetFileName(seg_fullname)
    png_writer.Update()
    png_writer.Write()
    
    ic.SetInputData(input_image)
    ic.Update()
    png_writer.SetInputData(ic.GetOutput())
    png_writer.SetFileName(img_fullname)
    png_writer.Update()
    png_writer.Write()
    
    sequence_browser_node.SelectNextItem()
    
    # slicer.app.processEvents()